In [2]:


### Import libraries

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
% matplotlib inline
import glob

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D,Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from scipy.ndimage.measurements import label
import time
import json
import numpy as np
import os


UsageError: Line magic function `%` not found.


In [3]:

batch_images = []
batch_masks = []

        
#### Testing generator, generates augmented images


In [8]:
from PIL import Image
path = 'train_annotations'

dir_list = os.listdir(path)

for inddir in dir_list:
    while(int(inddir[0:3])<50):

        maskedImagePath = 'train_videos/'+inddir[0:3]+"/maskedImages/"
        rawImagePath = 'train_videos/'+inddir[0:3]+"/rawImages/"

        for images in os.listdir(rawImagePath):
            img = Image.open(images)
            batch_images.append(np.asarray(img))
            if(len(batch_images)%100==0):
                print(batch_images)

        for masks in os.listdir(maskedImagePath):
            img = Image.open(masks)
            batch_masks.append(np.asarray(img))


FileNotFoundError: [Errno 2] No such file or directory: '42_raw.png'

In [5]:
batch_images

[array('42_raw.png', dtype='<U10'),
 array('157_raw.png', dtype='<U11'),
 array('82_raw.png', dtype='<U10'),
 array('57_raw.png', dtype='<U10'),
 array('88_raw.png', dtype='<U10'),
 array('34_raw.png', dtype='<U10'),
 array('140_raw.png', dtype='<U11'),
 array('21_raw.png', dtype='<U10'),
 array('65_raw.png', dtype='<U10'),
 array('132_raw.png', dtype='<U11'),
 array('126_raw.png', dtype='<U11'),
 array('55_raw.png', dtype='<U10'),
 array('89_raw.png', dtype='<U10'),
 array('47_raw.png', dtype='<U10'),
 array('137_raw.png', dtype='<U11'),
 array('62_raw.png', dtype='<U10'),
 array('11_raw.png', dtype='<U10'),
 array('79_raw.png', dtype='<U10'),
 array('29_raw.png', dtype='<U10'),
 array('45_raw.png', dtype='<U10'),
 array('113_raw.png', dtype='<U11'),
 array('40_raw.png', dtype='<U10'),
 array('121_raw.png', dtype='<U11'),
 array('151_raw.png', dtype='<U11'),
 array('123_raw.png', dtype='<U11'),
 array('98_raw.png', dtype='<U10'),
 array('70_raw.png', dtype='<U10'),
 array('125_raw.png

In [ ]:
def IOU_calc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return 2*(intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)


def IOU_calc_loss(y_true, y_pred):
    return -IOU_calc(y_true, y_pred)

In [ ]:
def get_small_unet():
    inputs = Input((img_rows, img_cols,3))
    inputs_norm = Lambda(lambda x: x/127.5 - 1.)
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    
    return model